In [10]:
#Loading the Dataset
import pandas as pd
from sklearn.model_selection import train_test_split

In [11]:
data = pd.read_csv('twitter_dataset.csv')
print(data.head())




   tweet_id     author_id  inbound                      created_at  \
0    119237        105834     True  Wed Oct 11 06:55:44 +0000 2017   
1    119238  ChaseSupport    False  Wed Oct 11 13:25:49 +0000 2017   
2    119239        105835     True  Wed Oct 11 13:00:09 +0000 2017   
3    119240  VirginTrains    False  Tue Oct 10 15:16:08 +0000 2017   
4    119241        105836     True  Tue Oct 10 15:17:21 +0000 2017   

                                                text response_tweet_id  \
0  @AppleSupport causing the reply to be disregar...            119236   
1  @105835 Your business means a lot to us. Pleas...               NaN   
2  @76328 I really hope you all change but I'm su...            119238   
3  @105836 LiveChat is online at the moment - htt...            119241   
4  @VirginTrains see attached error message. I've...            119243   

   in_response_to_tweet_id  
0                      NaN  
1                 119239.0  
2                      NaN  
3                 

In [9]:
# Preprocess the data

In [16]:
# Ensure 'tweet_id' and 'response_tweet_id' columns are of the same type
data['tweet_id'] = data['tweet_id'].astype(str)
data['response_tweet_id'] = data['response_tweet_id'].astype(str)

In [17]:
# Filter out necessary columns
data = data[['tweet_id', 'author_id', 'inbound', 'text', 'response_tweet_id']]

In [18]:
# Filter for customer queries (inbound=True) and support responses (inbound=False)
customer_queries = data[data['inbound'] == True]
support_responses = data[data['inbound'] == False]

In [19]:
# Merge customer queries with their corresponding responses
merged_data = customer_queries.merge(support_responses, left_on='tweet_id', right_on='response_tweet_id', suffixes=('_query', '_response'))


In [20]:
# Select relevant columns
merged_data = merged_data[['text_query', 'text_response']]

In [28]:
# Display the first few rows of the merged data
print(merged_data.head())

                                          text_query  \
0  @VirginTrains see attached error message. I've...   
1  @VirginTrains yep, I've tried laptop too sever...   
2  @VirginTrains I still haven't heard &amp; the ...   
3  @SpotifyCares Thanks! Version 8.4.22.857 armv7...   
4  @SpotifyCares No, but I've moved speaker to ab...   

                                       text_response  
0  @105836 LiveChat is online at the moment - htt...  
1  @105836 Have you tried from another device, Mi...  
2  @105836 That's what we're here for Miriam 😊  T...  
3  @105840 Hi there! What device is this happenin...  
4  @105840 Thanks. The distance could possibly af...  


In [ ]:
# Preprocessing and Generating Responses

In [26]:
import openai

openai.api_key = "sk-proj-kNkKP6PUdcS6ezBMcjW1T3BlbkFJaasbud6r4mTt8B8fTofO"

ModuleNotFoundError: No module named 'openai'

In [27]:
# Function to preprocess text data
def preprocess(text):
    return text.lower()

In [ ]:
# Apply preprocessing
merged_data['text_query'] = merged_data['text_query'].apply(preprocess)
merged_data['text_response'] = merged_data['text_response'].apply(preprocess)

In [29]:
# Function to generate a response using GPT-3.5
def generate_response(query):
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=f"Customer query: {query}\nSupport response:",
        max_tokens=150,
        temperature=0.7,
        top_p=1.0,
        frequency_penalty=0.5,
        presence_penalty=0.0
    )
    return response.choices[0].text.strip()

In [30]:
# Example usage
sample_query = merged_data.iloc[0]['text_query']
print(generate_response(sample_query))

NameError: name 'openai' is not defined

In [ ]:
# Flask API for Customer Queries

from flask import Flask, request, jsonify

app=Flask(twitter_customer_dataset)

@app.route('/generate-response', methods=['POST'])
def generate_response_api():
    data = request.json
    query = data['query']
    response = generate_response(query)
    return jsonify({"response": response})

if __name__ == '__main__':
    app.run(debug=True)
